In [1]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
using CSV
using JSON
using UUIDs
using Serialization
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

includet(dirname(pwd()) * "\\Water_Regulation\\WaterRegulation.jl")
using .WaterRegulation

### Simulations of Water Regulation Procedure

We would like to set up a skeleton for the day-to-day decision making of hydropower producers and the Water Regulation Company. The main steps of the Water Regulation company consist of:

* Determining a Reference Flow for the next nomination period (Inflow Forecasting)
* Receiving pre-bid nominations and calculating adjusted flow, power swaps, etc. -> Send these parameters back to the participants
* Recalculate based on renominations by power producers after bidding

For the Power Producers / Participants:  

* Fetch and Calculate all Parameters for the Water Regulation Procedure (Individual Reservoir Level etc.)
* Price and Inflow Forecasting (Fix Price Points)
* Medium-Term Hydropower Model Calculation, obtain Water Value Cuts
* Decision Model 1: Bidding and Pre-Market Clearing Nomination
* Decision Model 2: Short-Term Optimization and Renomination
* Decision Model 3: Fixed Water Flow scheduling and real time balancing

This constitutes one round of simulation.


In [2]:
filepath_Ljungan = dirname(pwd()) * "\\Water_Regulation\\TestDataWaterRegulation\\Ljungan.json"
filepath_prices = dirname(pwd()) *  "\\Data\\Spot Prices\\prices_df.csv"
filepath_results = dirname(pwd()) * "\\Results\\LambdaZero\\"
R, K, J = read_data(filepath_Ljungan)

println("The reservoir system consists of $([r.dischargepoint for r in R])")
println("Downstream of the reservoirs we have power plants $([k.name for k in K])")
for j in J
    println("$(j.name) owns $(j.plants), which constitutes a participation rate of: \n ", j.participationrate)
end


The reservoir system consists of ["Flasjon", "Holsmjon"]
Downstream of the reservoirs we have power plants ["Flasjo", "Trangfors", "Ratan", "Turinge", "Bursnas", "Jarnvagsforsen", "Parteboda", "Hermansboda", "Ljunga", "Nederede", "Skallbole", "Matfors", "Viforsen"]


Sydkraft owns HydropowerPlant[Flasjo, Trangfors, Ratan, Turinge, Bursnas], which constitutes a participation rate of: 
 

________________________________
Flasjon  | 1.84    
Holsmjon | 0.0     

Fortum owns HydropowerPlant[Parteboda, Hermansboda, Ljunga], which constitutes a participation rate of: 
 ________________________________
Flasjon  | 0.8200000000000001
Holsmjon | 0.8200000000000001

Statkraft owns HydropowerPlant[Nederede, Skallbole, Matfors, Viforsen, Jarnvagsforsen], which constitutes a participation rate of: 
 ________________________________
Flasjon  | 1.86    
Holsmjon | 1.86    



#### Load Data

Other than the river system, the relevant data for optimization is organized in Dataframes.
We load in cleaned versions of the data.

In [3]:
filepath_prices = dirname(pwd()) * "\\Inflow Forecasting\\Data\\Spot Prices\\prices_df.csv"
filepath_inflows = dirname(pwd()) * "\\Inflow Forecasting\\Data\\Inflow\\Data from Flasjoen and Holmsjoen.csv"

price_data = prepare_pricedata(filepath_prices)
inflow_data = prepare_inflowdata(filepath_inflows)

first(inflow_data, 10)

Row,Date,Flasjon Water Level,Flasjon Volume,Flasjon Inflow,Flasjon discharge,Holmsjon Water Level,Holmsjon Volume,Holmsjon Inflow,Holmsjon Discharge,CalendarWeek,year
,Date,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Int64,Int64
1,2006-01-01,484.0,2087.0,12.8,49.0,244.39,1928,13.6,53.2,52,2006
2,2006-01-02,483.83,2056.0,13.6,49.0,244.42,1947,22.7,89.1,1,2006
3,2006-01-03,483.69,2031.0,10.2,49.0,244.41,1941,13.4,85.9,1,2006
4,2006-01-04,483.52,2000.0,10.2,50.0,244.41,1941,23.0,88.1,1,2006
5,2006-01-05,483.34,1968.0,8.2,52.0,244.41,1941,15.0,86.2,1,2006
6,2006-01-06,483.14,1932.0,7.9,52.0,244.4,1934,19.1,74.2,1,2006
7,2006-01-07,482.94,1896.0,6.2,53.0,244.41,1941,16.0,84.2,1,2006
8,2006-01-08,482.73,1860.0,11.2,53.0,244.4,1934,22.0,85.2,1,2006
9,2006-01-09,482.53,1826.0,2.6,53.0,244.39,1928,-0.3,104.2,2,2006


### Price Forecasting

Price is sensitive to various factors. Daytime, Weekday and time of year play a relevant role in how prices usually behave.

* For the bidding and short-term optimization problems, part of the uncertainty lies in unknown hourly prices.

* For the medium term problem, we are interested in daily (average) prices.

* We want to obtain Price Points which will be important in the analysis from the historic data.

* We need penalty parameters for using the balancing markets, which could be obtained from using maximum / minimum values fromm observed spot market prices / generated scenarios.

In [12]:
const scenario_count_prices_short = 10
const scenario_count_prices_medium = 2
const stage_count_short = 8
const stage_count_medium = 52
const price_point_count = 5
const T = 24

PriceScenariosMedium = Price_Scenarios_Medium(price_data, scenario_count_prices_medium)
PriceScenariosShort = Price_Scenarios_Short(price_data, scenario_count_prices_short, stage_count_short)
PPoints = Create_Price_Points(price_data, 5)
mu_up, mu_down = BalanceParameters(PriceScenariosShort)

(93.08, 5.0)

### Inflow Forecasting

Inflow has extreme seasonal differences, depending on geographical location.  
For the Ljungan River System we could observe that most inflow occurs during the spring melt around May.

In [5]:
const ColumnReservoir = Dict{Reservoir, String}(R[1] => "Flasjon Inflow", R[2] => "Holmsjon Inflow")
const scenario_count_inflows_weekly = 2
const scenario_count_inflows_short = 1


InflowScenariosShort = Inflow_Scenarios_Short(inflow_data, 1, ColumnReservoir, R, stage_count_short, scenario_count_prices_short)
InflowScenariosMedium = Inflow_Scenarios_Medium(inflow_data, ColumnReservoir, scenario_count_inflows_weekly, R)
first(InflowScenariosShort, 10)

8-element Vector{Pair{Int64, Dict{Reservoir, Vector{Float64}}}}:
 5 => Dict(Flasjon => [20.95, 6.34, 6.67, 16.2, 5.8, 11.6, 7.48, 15.3, 13.49, 11.3], Holsmjon => [32.9, 17.4, 15.0, 22.0, 15.2, 21.6, 2.2, 32.9, 32.9, 12.2])
 4 => Dict(Flasjon => [8.18, 6.58, 8.6, 8.2, 20.95, 8.21, 7.35, 11.6, 15.3, 12.86], Holsmjon => [4.0, 38.9, 0.0, 54.6, 29.9, 43.9, 54.8, 49.7, 0.0, 0.0])
 6 => Dict(Flasjon => [5.18, 10.63, 7.2, 7.35, 15.3, 1.88, 7.4, 7.8, 15.4, 20.95], Holsmjon => [28.3, 49.7, 25.7, 77.5, 14.5, 0.0, 9.2, 26.2, 49.7, 13.9])
 7 => Dict(Flasjon => [7.25, 10.2, 5.33, 5.79, 10.87, 9.71, 7.62, 10.52, 15.4, 7.2], Holsmjon => [24.4, 26.4, 4.7, 12.2, 12.2, 21.5, 48.1, 23.5, 11.7, 20.0])
 2 => Dict(Flasjon => [0.0, 8.21, 6.05, 5.33, 8.21, 8.91, 2.51, 6.34, 6.05, 5.32], Holsmjon => [0.0, 16.0, 16.1, 33.6, 0.0, 4.1, 20.4, 22.7, 20.4, 0.0])
 8 => Dict(Flasjon => [5.8, 7.2, 5.33, 8.41, 6.6, 9.92, 1.88, 9.05, 6.82, 10.43], Holsmjon => [11.0, 57.8, 0.0, 29.9, 14.5, 0.0, 0.0, 48.7, 33.2, 0.0])
 3 =>

### Create Uncertainty Set

Depending on the Problem, we create the uncertainty sets from both price and inflow Scenarios.
This becomes more difficult for the Anticipatory Bidding Problem.

In [13]:
Ω_medium, P_medium =  create_Ω_medium(PriceScenariosMedium, InflowScenariosMedium, R);
Ω_NA, P_NA = create_Ω_Nonanticipatory(PriceScenariosShort, InflowScenariosShort, R, stage_count_short)
Ω_NA[1]

"""
    create_Ω_Anticipatory(PriceScenariosShort, InflowScenariosShort, j, R)

    create Uncertainty Set for the Anticipatory Problem: 
    Addiitionally to the Nonanticipatory Case, we associate deterministic sets of inflows and prices with other producers' nominations.
    The Uncertainty set is extended entrywise by the others' nomination. 
"""

function create_Ω_Anticipatory(PriceScenariosShort, InflowScenariosShort, OthersNominations, j::Participant, R::Vector{Reservoir}, stage_count = stage_count_bidding)
    Ω = Dict(i => [(inflow = Dict(r => InflowScenariosShort[i][r][j] for r in R), price = c) for c in PriceScenariosShort[i] for j in eachindex(InflowScenariosShort[i][R[1]])] for i in 1:stage_count)
    P = Dict(s => [1/length(eachindex(Ω[s])) for i in eachindex(Ω[s])] for s in 1:stage_count)
    return Ω, P
end
# Ω_A, P_A = create_Ω_Anticipatory()

create_Ω_Anticipatory (generic function with 2 methods)

### Reference Flow

To calculate a reference flow for each reservoir, the Water Regulation Company has to combine information from inflow, historical reservoir trajectories and current reservoir level.  

In [14]:
l_traj, f = AverageReservoirLevel(R, inflow_data)
Qref = CalculateReferenceFlow(R, l_traj, f, 2)

Dict{Reservoir, Float64} with 2 entries:
  Flasjon  => -97.0201
  Holsmjon => -80.76

## Participants

Every Participant individually plans their bidding and production. Irrespective of the strategy, only limited information is available. Internal to the participants problems are 

* Price Forecasts
* Nomination to Water Regulation Company
* Bidding at Electricity Market

### Medium Term Problem

The Medium Term / Seasonal Optimization Problem is necessary to obtain Cuts for the Water Value Function.

Therefor the point of a functioin containing this decision model is to return cut coefficients of the Water Value Function.

* What are the inputs to the Medium Term problem?
* How are the cuts calculated?
* How many cuts should be calculated?

In [15]:
const iterations = 10

const savepath_watervalue = "C:\\Users\\lenna\\OneDrive - NTNU\\Code Master Thesis\\Inflow Forecasting\\WaterValue"

WaterValueDictionary_j, WaterValueDictionary_O = MediumModelsAllParticipants(J, R, Ω_medium, P_medium, stage_count_medium, iterations)
println(WaterValueDictionary_j)
println(WaterValueDictionary_O)
SaveMediumModel(savepath_watervalue * "\\Participant.jls", WaterValueDictionary_j)
SaveMediumModel(savepath_watervalue * "\\OtherParticipant.jls", WaterValueDictionary_O)

  - bounds range contains large coefficients
Very large or small absolute values of coefficients
can cause numerical stability issues. Consider
reformulating the model.


  - bounds range contains large coefficients
Very large or small absolute values of coefficients
can cause numerical stability issues. Consider
reformulating the model.


Dict{Participant, SDDP.PolicyGraph}(Statkraft => A policy graph with 52 nodes.


 Node indices: 1, ..., 52
, Sydkraft => A policy graph with 52 nodes.
 Node indices: 1, ..., 52
, Fortum => A policy graph with 52 nodes.
 Node indices: 1, ..., 52
)
Dict{Participant, SDDP.PolicyGraph}(Statkraft => A policy graph with 52 nodes.
 Node indices: 1, ..., 52
, Sydkraft => A policy graph with 52 nodes.
 Node indices: 1, ..., 52
, Fortum => A policy graph with 52 nodes.
 Node indices: 1, ..., 52
)


In [24]:
NameToParticipant = Dict{String, Participant}(j.name => j for j in J)
MediumModelDictionary_j_loaded = ReadMediumModel(savepath_watervalue * "\\Participant.jls", NameToParticipant);
MediumModelDictionary_O_loaded = ReadMediumModel(savepath_watervalue * "\\OtherParticipant.jls", NameToParticipant);


cuts = Dict(j => ReservoirLevelCuts(R, j.plants, j, f, 2, stage_count_short) for j in J)
WaterCuts = Dict(j => WaterValueCuts(MediumModelDictionary_j_loaded[j], cuts[j], 2) for j in J)

Reservoir[Flasjon, Holsmjon]
Dict{Reservoir, Float64}[________________________________
Flasjon  | 1149.471746031746
, ________________________________
Flasjon  | 1080.0  
, ________________________________
Flasjon  | 2069.471746031746
, ________________________________
Flasjon  | 2000.0  
]Reservoir[Flasjon, Holsmjon]


Dict{Reservoir, Float64}[________________________________
Flasjon  | 2000.0  
Holsmjon | 0.0     
, ________________________________
Flasjon  | 909.4717460317461
Holsmjon | 0.0     
, ________________________________
Flasjon  | 2069.471746031746
Holsmjon | 0.0     
, ________________________________
Flasjon  | 909.4717460317461
Holsmjon | 945.0857142857143
, ________________________________
Flasjon  | 909.4717460317461
Holsmjon | 0.0     
, ________________________________
Flasjon  | 2000.0  
Holsmjon | 800.0   
, ________________________________
Flasjon  | 909.4717460317461
Holsmjon | 800.0   
, ________________________________
Flasjon  | 840.0   
Holsmjon | 0.0     
, ________________________________
Flasjon  | 2000.0  
Holsmjon | 0.0     
, ________________________________
Flasjon  | 840.0   
Holsmjon | 800.0   
, ________________________________
Flasjon  | 840.0   
Holsmjon | 945.0857142857143
, ________________________________
Flasjon  | 2069.471746031746
Holsmjon | 945.0857142857

Dict{Participant, Dict{Dict{Reservoir, Float64}, NamedTuple{(:e1, :e2), Tuple{Float64, Dict{Symbol, Float64}}}}} with 3 entries:
  Statkraft => Dict(________________________________…
  Sydkraft  => Dict(________________________________…
  Fortum    => Dict(________________________________…

### Bidding Problem

* Every Power Producer solves their own Bidding problem with individual Water Value Cuts to obtain their bidding curves and nomination.
* The solutions are taken as input for the calculation of the adjusted Flow and Power Swap
* The Market clears, and the obligations for the next production day are revealed.

In [29]:
HourlyBiddingCurves = []
Qnoms = []

for j in J
    Qnom, HourlyBiddingCurve = Nonanticipatory_Bidding(R, j, PPoints, Ω_NA, P_NA, Qref, cuts[j], WaterCuts[j], iterations, mu_up, mu_down, T, stage_count_short)
    push!(Qnoms, Qnom)
    push!(HourlyBiddingCurves, HourlyBiddingCurve)
end

Qnoms

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 8
  state variables : 176
  scenarios       : 1.00000e+14
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                                                                   : [356, 813]
  AffExpr in MOI.EqualTo{Float64}                                               : [7, 61]
  AffExpr in MOI.GreaterThan{Float64}                                           : [115, 115]
  AffExpr in MOI.LessThan{Float64}                                              : [148, 388]
  VariableRef in MOI.GreaterThan{Float64}                                       : [171, 389]
  VariableRef in MOI.LessThan{Float64}                                          : [171

numerical stability report
  matrix range     [1e-04, 2e+02]
  objective range  [1e-02, 4e+02]
  bounds range     [7e+00, 5e+06]
  rhs range        [1e+02, 3e+05]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-------------------------------------------------------------------


         1   1.664323e+06  2.266847e+06  1.965000e+00       709   1


         2   5.991755e+05  1.838080e+06  4.140000e+00      1418   1


         3   1.717057e+06  1.756259e+06  6.336000e+00      2127   1


...

         3   4.574851e+06  5.000436e+06  8.203000e+00      2127   1


         6   4.398108e+06  4.860752e+06  1.557600e+01      4254   1


         9   4.352043e+06  4.819475e+06  2.327500e+01      6381   1


        10   3.572788e+06  4.805091e+06  2.587700e+01      7090   1
-------------------------------------------------------------------
status         : iteration_limit
total time (s) : 2.587700e+01
total solves   : 7090
best bound     :  4.805091e+06
simulation ci  :  4.286904e+06 ± 5.036681e+05
numeric issues : 0
-------------------------------------------------------------------



3-element Vector{Any}:
 ________________________________
Flasjon  | 63.720606556273424

 ________________________________
Flasjon  | 0.0     
Holsmjon | 132.46229726069203

 ________________________________
Flasjon  | 0.0     
Holsmjon | 106.61392339763222


### Water Regulation

Without Updating the Reservoir Levels, the adjusted flow and Power Swaps are calculated.

### Short Term Optimization

### Real Time Balancing and Scheduling

# Simulation

For a preset amount of rounds, we simulate the reservoir system through the functions and structures as described above. It is important to have a way to save results adequately! This encapsulates the calculations, and evaluations and visualizations can take place at a later point in time.